In [4]:
from MAUtil import *
from MAInit import *
from MACalc import *
from MAPost import *

In [5]:
### get file names ###
env = 'local'
files = os.listdir('./database/')
filenames = []

for i in range(len(files)):
    if 'no' in files[i]:
        if '__' not in files[i]:
            if 'all' not in files[i]:
                filenames.append(files[i])
                print(i, files[i])

18 Cu_100_u2_BEEF-vdW_no001_CO_n1_d8.traj
22 Cu_100_u2_BEEF-vdW_no010_CO_n1_d8.traj
26 Cu_100_u2_BEEF-vdW_no019_CO_n1_d8.traj
32 Cu_100_u2_RPBE-D2_no001_CO_n1_d8.traj
36 Cu_100_u2_RPBE-D2_no010_CO_n1_d8.traj
40 Cu_100_u2_RPBE-D2_no020_CO_n1_d8.traj
46 Cu_100_u2_RPBE_no001_CO_n1_d8.traj
50 Cu_100_u2_RPBE_no010_CO_n1_d8.traj
54 Cu_100_u2_RPBE_no020_CO_n1_d8.traj
60 Cu_100_u2_optB88-vdW_no001_CO_n1_d8.traj
64 Cu_100_u2_optB88-vdW_no010_CO_n1_d8.traj
68 Cu_100_u2_optB88-vdW_no020_CO_n1_d8.traj
75 Cu_100_u2_vdW-DF2_no001_CO_n1_d8.traj
79 Cu_100_u2_vdW-DF2_no010_CO_n1_d8.traj
83 Cu_100_u2_vdW-DF2_no020_CO_n1_d8.traj
88 Cu_100_u2_vdW-DF_no001_CO_n1_d8.traj
92 Cu_100_u2_vdW-DF_no010_CO_n1_d8.traj
96 Cu_100_u2_vdW-DF_no020_CO_n1_d8.traj
102 Cu_111_u2_BEEF-vdW_no001_CO_n1_d8.traj
106 Cu_111_u2_BEEF-vdW_no008_CO_n1_d8.traj
110 Cu_111_u2_BEEF-vdW_no015_CO_n1_d8.traj
114 Cu_111_u2_BEEF-vdW_no020_CO_n1_d8.traj
120 Cu_111_u2_RPBE-D2_no001_CO_n1_d8.traj
124 Cu_111_u2_RPBE-D2_no008_CO_n1_d8.traj
128 Cu

In [6]:
for file in filenames:
    try:
        A = make_database(file)
        A.delete_from_database()
    except:
        print(file, 'Some error occured.')

-----------------------------------------------------------
Not in database.
-----------------------------------------------------------
Not in database.
-----------------------------------------------------------
Not in database.
-----------------------------------------------------------
Not in database.
-----------------------------------------------------------
Not in database.
-----------------------------------------------------------
Not in database.
-----------------------------------------------------------
Not in database.
-----------------------------------------------------------
Not in database.
-----------------------------------------------------------
Not in database.
-----------------------------------------------------------
Not in database.
-----------------------------------------------------------
Not in database.
-----------------------------------------------------------
Not in database.
-----------------------------------------------------------
Not in database.

In [7]:
for file in filenames:
    try:
        A = make_database(file)
    except:
        print(file, 'Some error occured.')
    A.add_to_database()
    A.update_Energy()
    A.check_database()

-----------------------------------------------------------
Not in database.
Cu_100_u2_BEEF-vdW_no001_CO_n1_d8.traj
One post: 5e5ef52497ec0a1505ba519f

This function is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Not in database.
Cu_100_u2_BEEF-vdW_no010_CO_n1_d8.traj
One post: 5e5ef52497ec0a1505ba51a1

This function is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Not in database.
Cu_100_u2_BEEF-vdW_no019_CO_n1_d8.traj
One post: 5e5ef52597ec0a1505ba51a3

This function is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Not in database.
Cu_100_u2_RPBE-D2_no001_CO_n1_d8.traj
One post: 5e5ef52597ec0a1505ba51a5

This function is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Not in database.
Cu_100_u2_RPBE-D2_no010_CO_n1_d8.traj
One post: 5e5ef52697ec0a1505ba51a7

This funct

Cu_111_u2_vdW-DF_no008_CO_n1_d8.traj
One post: 5e5ef53697ec0a1505ba51ed

This function is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Not in database.
Cu_111_u2_vdW-DF_no015_CO_n1_d8.traj
One post: 5e5ef53697ec0a1505ba51ef

This function is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Not in database.
Cu_111_u2_vdW-DF_no020_CO_n1_d8.traj
One post: 5e5ef53797ec0a1505ba51f1

This function is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Not in database.
Pd_111_u1_RPBE_no001_CO_n1_d3.traj
One post: 5e5ef53797ec0a1505ba51f3

This function is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Not in database.
Pd_111_u1_RPBE_no002_CO_n1_d3.traj
One post: 5e5ef53797ec0a1505ba51f5

This function is for surface with more than 2 adsorbates.
-----------------------------------------

Pt_100_u2_BEEF-vdW_no010_CO_n1_d9.traj
One post: 5e5ef55e97ec0a1505ba5237

This function is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Not in database.
Pt_100_u2_BEEF-vdW_no020_CO_n1_d9.traj
One post: 5e5ef55e97ec0a1505ba5239

This function is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Not in database.
Pt_100_u2_RPBE-D2_no001_CO_n1_d8.traj
One post: 5e5ef55f97ec0a1505ba523b

This function is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Not in database.
Pt_100_u2_RPBE-D2_no010_CO_n1_d8.traj
One post: 5e5ef55f97ec0a1505ba523d

This function is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Not in database.
Pt_100_u2_RPBE-D2_no019_CO_n1_d8.traj
One post: 5e5ef56097ec0a1505ba523f

This function is for surface with more than 2 adsorbates.
------------------------------

Pt_111_u2_vdW-DF_no020_CO_n1_d9.traj
One post: 5e5ef57197ec0a1505ba5287

This function is for surface with more than 2 adsorbates.


In [8]:
for file in filenames:
    try:
        A = make_database(file)
        data = A.check_database()
        print(data['name'], data['converged'], data['not_moved'], data['kept_sites'], data['isvalid'], data['totaladsE'])
    except:
        print(file, 'Some error occured.')

-----------------------------------------------------------
Cu_100_u2_BEEF-vdW_no001_CO_n1_d8.traj no yes yes no -0.59465535
-----------------------------------------------------------
Cu_100_u2_BEEF-vdW_no010_CO_n1_d8.traj yes yes yes yes -0.3970746700000021
-----------------------------------------------------------
Cu_100_u2_BEEF-vdW_no019_CO_n1_d8.traj yes yes yes yes -0.5161128500000025
-----------------------------------------------------------
Cu_100_u2_RPBE-D2_no001_CO_n1_d8.traj no yes yes no -0.7787489500000078
-----------------------------------------------------------
Cu_100_u2_RPBE-D2_no010_CO_n1_d8.traj yes yes yes yes -0.7644200699999999
-----------------------------------------------------------
Cu_100_u2_RPBE-D2_no020_CO_n1_d8.traj yes yes yes yes -0.698101390000005
-----------------------------------------------------------
Cu_100_u2_RPBE_no001_CO_n1_d8.traj yes yes yes yes -0.5497481199999967
-----------------------------------------------------------
Cu_100_u2_RPBE_

Pd_111_u3_RPBE_no003_CO_n1_d14.traj yes yes yes yes -1.8417528999999888
-----------------------------------------------------------
Pd_111_u3_RPBE_no004_CO_n1_d14.traj yes yes yes yes -1.8199562300000025
-----------------------------------------------------------
Pd_111_u4_RPBE_no002_CO_n1_d20.traj yes yes yes yes -1.5229765500000099
-----------------------------------------------------------
Pd_111_u4_RPBE_no004_CO_n1_d20.traj yes yes yes yes -1.680979050000019
-----------------------------------------------------------
Pt_100_u2_BEEF-vdW_no001_CO_n1_d9.traj yes yes yes yes -1.622475080000001
-----------------------------------------------------------
Pt_100_u2_BEEF-vdW_no010_CO_n1_d9.traj yes yes yes yes -1.7396392300000016
-----------------------------------------------------------
Pt_100_u2_BEEF-vdW_no020_CO_n1_d9.traj yes yes yes yes -1.1979731200000003
-----------------------------------------------------------
Pt_100_u2_RPBE-D2_no001_CO_n1_d8.traj yes yes yes yes -1.904150669999

In [9]:
for file in filenames:
    try:
        A = make_database(file)
        A.update_adsorbates_correlation()
    except:
        print(file, 'Some error occured.')

-----------------------------------------------------------
Cu_100_u2_BEEF-vdW_no001_CO_n1_d8.traj Adsorbate correlation updated.
-----------------------------------------------------------
Cu_100_u2_BEEF-vdW_no010_CO_n1_d8.traj Adsorbate correlation updated.
-----------------------------------------------------------
Cu_100_u2_BEEF-vdW_no019_CO_n1_d8.traj Adsorbate correlation updated.
-----------------------------------------------------------
Cu_100_u2_RPBE-D2_no001_CO_n1_d8.traj Adsorbate correlation updated.
-----------------------------------------------------------
Cu_100_u2_RPBE-D2_no010_CO_n1_d8.traj Adsorbate correlation updated.
-----------------------------------------------------------
Cu_100_u2_RPBE-D2_no020_CO_n1_d8.traj Adsorbate correlation updated.
-----------------------------------------------------------
Cu_100_u2_RPBE_no001_CO_n1_d8.traj Adsorbate correlation updated.
-----------------------------------------------------------
Cu_100_u2_RPBE_no010_CO_n1_d8.traj Ad

Pd_111_u2_RPBE_no019_CO_n2_d4.traj Adsorbate correlation updated.
-----------------------------------------------------------
Pd_111_u2_RPBE_no020_CO_n1_d9.traj Adsorbate correlation updated.
-----------------------------------------------------------
Pd_111_u2_RPBE_no021_CO_n2_d3.traj Adsorbate correlation updated.
-----------------------------------------------------------
Pd_111_u2_RPBE_no022_CO_n3_d3.traj Adsorbate correlation updated.
-----------------------------------------------------------
Pd_111_u2_RPBE_no023_CO_n4_d3.traj Adsorbate correlation updated.
-----------------------------------------------------------
Pd_111_u3_RPBE_no001_CO_n1_d14.traj Adsorbate correlation updated.
-----------------------------------------------------------
Pd_111_u3_RPBE_no002_CO_n1_d14.traj Adsorbate correlation updated.
-----------------------------------------------------------
Pd_111_u3_RPBE_no003_CO_n1_d14.traj Adsorbate correlation updated.
-------------------------------------------------

In [ ]:
for file in filenames:
    try:
        A = make_database(file)
        data = A.check_database()
        print(data['name'], data['minimum_distance'], data['ads_dist2'], data['ads_dist3'])
    except:
        print(file, 'Some error occured.')

In [8]:
# for file in filenames:
#     try:
#         A = make_database(file)
#         print(A.check_database()['minimum_distance'])
#     except:
#         print(file, 'Some error occured.')

In [7]:
file = 'Pd_111_u2_RPBE_no016_CO_n2_d3.traj'
A = make_database(file)
A.check_database()

-----------------------------------------------------------


{'_id': ObjectId('5e5e9294ba99fcf946db28bd'),
 'name': 'Pd_111_u2_RPBE_no016_CO_n2_d3.traj',
 'isvalid': 'yes',
 'element': 'Pd',
 'face': '111',
 'unitlength': 2,
 'xc': 'RPBE',
 'adsorbate': 'CO',
 'numberofads': 2,
 'coverage': 0.5,
 'surfatomnum': 4,
 'E': -101.78655052,
 'bareE': -70.33920809,
 'E_CO': -14.48543631,
 'totaladsE': -2.4764698100000118,
 'aveadsE/suratom': -0.6191174525000029,
 'aveadsE/ads': -1.2382349050000059,
 'E_int_space': 0.42124055000000027,
 'sumE_each_ads': -3.178556780000008,
 'E_residue': 0.07021160499999901,
 'area': 27.42573780073108,
 'density': 0.07292420041829054,
 'igroups': [2, 2],
 'rgroups': [2, 2],
 'COlengthlis': [1.1869709481800297, 1.1870194222378032],
 'converged': 'yes',
 'not_moved': 'yes',
 'kept_sites': 'yes',
 'E_not_exceeded': 'yes',
 'minimum_distance': 2,
 'ads_dist2': 0.4444444444444444,
 'ads_dist3': 0.24691358024691357}

In [22]:
# file = 'Cu_100_u2_BEEF-vdW_no001_CO_n1_d8.traj'
init_query(file, 'local')
atoms = query('Cu_100_u2_vdW-DF_no010_CO_n1_d8_all.traj', 'local')
np.max(np.abs(atoms.get_forces()))

0.01037519